# Abstract

In this project, we present a mathematical model for optimizing 
road construction scheduling, focusing on a main road under 
construction and a detour used to alleviate congestion. The 
objective is to complete required construction in as short a time as posssible while minimizing added travel time during construction. We use Greenshields equations to model traffic velocity and Pontryagin's Maximum Principle to determine the optimal 
construction strategy.

# Background

As a construction planner, you need to decide on what 
schedule to conduct road construction in Utah. You know that 
your projects significantly impact traffic congestion. The 
question is how to optimize the construction schedule to 
minimize disruptions for drivers while maximizing progress on 
your projects.

We investigate a simple version of this problem where we onle consider two roads: the main 
road experiencing construction and alternate detour that is 
50% longer. The construction on the main road reduces its 
maximum density $P_1$
, influencing traffic. We identify $\Omega$ as the 
amount of construction required to complete the project.

# Mathematical Representation

Probably a mostly analytical/markdowny section with a bunch of latex laying out the math

### Cost Functional/Constraints

### Pontryagin's Maximum Principle 

### Gradient

# Solution

### Code for Ian's iterative solver thing

In [1]:
#Import stuff
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

In [ ]:
#Solve this for 0 using scipy.root and constrain to boundaries 
def DH_DP1(P1,P2,Drho1_DP1_x0, Drho2_DP1_x0, Drho1_DP1_xf, Drho2_DP1_xf, rho1_x0,rho2_x0, rho1_xf, rho2_xf,V1,V2,omega1,omega2,omega3,):
    term1 = omega1*V1*(Drho1_DP1_x0*(1-rho1_x0/P1)-rho1_x0*(Drho1_DP1_x0/P1-rho1_x0/P1**2)
                        -Drho1_DP1_xf*(1-rho1_xf/P1)+rho1_xf*(Drho1_DP1_xf/P1-rho1_xf/P1**2))
    term2 = omega2*V2*(Drho2_DP1_x0*(1-2*rho2_x0/P2) - Drho1_DP1_xf*(1-2*rho2_x0/P2))

    return term1 + term2 - omega3

def Drho1and2_DP1(P1,P2, J, V1, V2, epsilon=0.5):
    #Define some stuff for notational convenience
    D1 = P1*P2*(V1+V2)
    D2 = V1*P2-P1*V2*(1+epsilon)

    term1 = (P2*(V1+V2)/D2)+(D1*V2*(1+epsilon)/D2)
    term2 = ((J+D1/D2)**2 - 4*P1*P2*V1*J/D2)**(-0.5)
    term3 = 2*(J+D1/D2)*(P2*(V1+V2)/D2+D1*V2*(1+epsilon)/D2)
    term4 = 4*P2*V1*J/D2 + 4*P1*P2*V2*(V1+V2)*(1+epsilon)/D2

    #0.5*(1+epsilon)*(-(P2*(V1+V2)/D2))
    ret = 0.5*(term1 - term2*(term3 - term4))

    return ret, (1+epsilon)*ret

def omega3(P1,rho1,rho2,C2=1, Omega=10):
    -2*C2*(N3-Omega)

In [2]:
@np.vectorize
def J_T(t2):
    #Allow t to be a float
    day = t2 // 24 #Get an integer corresponding to the day of week: Monday=0, Sunday=6
    hour = t2 - 24*day #Get the time of day in military time
    day = day%7 #For longer-than-a-week timescales

    if day < 5:
      scale = 1.
    if day >= 5:
      scale = 0.4
    if hour < 5:
      return 0
    if 5 <= hour <= 14:
      return scale*(hour-5)/float(14-5)
    if 14 <= hour:
      return scale*(1-(hour-14)/float(24-14))

In [3]:
n = 50
dx = 1./n
eps = 0.5
V1 = 50
V2 = 50/(1+eps)
P2 = 50*(1+eps)
delta = .001 #Or P2
P_max = P2/(1+eps)

C1 = 1

def ode(t, rho, p): #Rho is a space-discretization of rho_1 and rho_2: n subintervals of each
  #The ordering is all the rho_1s, followed by all the (n+1) rho_2s, so there are 2n+2 values indexed from 0 to 2n+1
  #p[0] is tf

  #Define flux at x=0
  #Time in terms of hours since beginning of week (12am between Sunday and Monday is 0)

  # rho[rho > P_max] = P_max

  @np.vectorize
  def J_T(t2):
    #Allow t to be a float
    day = t2 // 24 #Get an integer corresponding to the day of week: Monday=0, Sunday=6
    hour = t2 - 24*day #Get the time of day in military time
    day = day%7 #For longer-than-a-week timescales

    if day < 5:
      scale = 1
    if day >= 5:
      scale = 0.4
    if hour < 5:
      return 0
    if 5 <= hour <= 14:
      return scale*(hour-5)/(14-5)
    if 14 <= hour:
      return scale*(1-(hour-14)/(24-14))

  # print(J_T(np.array([1,2])))

  P1 = np.expand_dims(np.sqrt((1-t)*(V1*(rho[n])**2 + V2*(rho[-1])**2)), 0) #Because we replace t with p[0] here, the division by tf cancels
  # print(P1.shape)
  deltas = np.ones_like(P1)*delta
  P1 = np.max(np.concatenate((P1, deltas)), axis=0)
  # P_maxes = np.ones_like(P1)*P2/(1+eps)
  # # print(P1.shape)
  # # print(P1)
  # P1 = np.min(np.concatenate((P1, P_maxes)), axis=0)
  # print(P1.shape)

  #Boundary conditions: Just force rho to the appropriate value here?
  #Define x=0 conditions on rho

  # # @np.vectorize
  # def rho_bound(t3): #rho 1 and 2 at t=t, x=0
  #   J = J_T(t3)
  #   print
  #   d3 = -2*P1*P2*(V1**2) - 2*P1*P2*V1*V2 - P2*(V1**2)*J - P1*V1*V2*J
  #   # print(f"checkpoint {t}")
  #   d2 = 2*(P1**2)*P2*((V1+V2)**2) + P1*(4*P2*(V1**2)+3*P2*V1*V2 + P1*V1*V2 + 2*P1*(V2**2))*J + V1*(P2*V1+P1*V2)*(J**2)
  #   d1 = -3*(P1**2)*P2*V1*(V1+V2)*J - P1*V1*(2*P2*V1 + P1*V2)*(J**2)
  #   d0 = (P1**2)*P2*(V1**2)*(J**2)
  #   D1 = 2*(d2**3) - 9*(d3*d2*d1) + 27*(d3**2)*d0
  #   D0 = (d2**2) - 3*d3*d1

  #   # assert D1**2 - 4*(D0**3) < 0

  #   # D2 = (((-1 + np.sqrt(3)*(0+1j))**2)/4)*(D1 + np.emath.sqrt(D1**2 - 4*(D0**3))/2)**(1/3)
  #   rho1_bound = -(d2+2*np.sqrt(D0)*np.cos(4*np.pi/3 + np.arccos(D1/(2*D0**1.5))/3))/(3*d3)
  #   # rho1_bound = np.real(-1*(d2+D2+D0/D2)/(3*d3))
  #   rho2_bound = (1+eps)*(J - rho1_bound)
  #   # print(rho1_bound.shape)
  #   # print(rho2_bound.shape)
  #   return rho1_bound, rho2_bound

  

  #Enforce them boundary conditions on entering traffic
  # print(type(rho_bound(np.array([1,2]))))
  # print(t*p[0])
  # print(type(rho_bound(np.array([0,1]))))
  rho1_bound, rho2_bound = rho_bound(t*p[0])
  assert rho1_bound.shape == rho[0].shape
  rho[0] = rho1_bound
  rho[n+1] = rho2_bound

  d_rho_dx = np.empty_like(rho)
  d_rho_dx[0] = (rho[1]-rho[0])/dx #I'm not 100% confident this is the best way to handle endpoints, but here we are for now
  d_rho_dx[1:n] = (rho[2:n+1] - rho[:n-1])/(2*dx)
  d_rho_dx[n] = (rho[n] - rho[n-1])/dx
  d_rho_dx[n+1] = (rho[n+2]-rho[n+1])/dx
  d_rho_dx[n+2:-1] = (rho[n+3:] - rho[n+1:-2])/(2*dx)
  d_rho_dx[-1] = (rho[-1] - rho[-2])/dx

  d_rho_dt = np.empty_like(d_rho_dx)
  d_rho_dt[:n+1] = -V1*d_rho_dx[:n+1]*(1-2*rho[:n+1]/P1)
  d_rho_dt[n+1:] = -V2*d_rho_dx[n+1:]*(1-2*rho[n+1:]/P2)

  assert d_rho_dt.shape == rho.shape
  return d_rho_dt

In [4]:
#Define initial conditions
init_state = np.zeros(2*n+2)/10. #Agrees with space-boundary condition

delta = .001 #Or P2
#Set up boundary conditions
def bcs(ya, yb, p):
  #Assume that at tf, C3 = 0 by the condition on it (because P1 is small and positive)
  num_cars = np.sum(yb)*dx
  P1_tf = delta
  return np.concatenate((ya - init_state, np.array([num_cars + C1])))

In [5]:
#Solve, using guesses
t_steps = 200
t = np.linspace(0,1,t_steps)
y0 = np.ones((2*n+2,t_steps))*.01
p0 = np.array([168])
res = solve_bvp(ode, bcs, t, y0, p0, max_nodes = 360000)

/tmp/ipykernel_283544/54886252.py:69: RuntimeWarning: invalid value encountered in arccos
  rho1_bound = -(d2+2*np.sqrt(D0)*np.cos(4*np.pi/3 + np.arccos(D1/(2*D0**1.5))/3))/(3*d3)


### Method of Characteristics

### Ian's Code

### PyClaw

One key problem we ran into during the above analysis is the intersection of characteristic curves. Since each curve has constant but potentially distinct density, solutions to the problem aren't defined in the traditional sense at the intersection of curves. However, weak solutions can be found that include "shocks" or discontinuities at these points. The precise derivation used by the PyClaw package's Riemann solver for Burger's equation is the one given by Lawrence C. Evans's *Partial Differential Equations.* The Rankine-Hugoniot condition states that if the PDE
$$
u_t + (F(u))_x = 0
$$
has a solution which is smooth except along some curve $C$, then along the curve $C$ we have
$$
F(u_l) - F(u_r) = \dot s (u_l - u_r)
$$
where $u_l$ is the limit of the solution to the left of the curve $C$, $u_r$ is the limit from the right, and $\dot s$ is the time derivative of any parametrization $x = s(t)$ of the curve $C$. 

Rather than taking the time necessary to code up our own solver for this method, we relied on PyClaw's implementation for Burger's equation. However, PyClaw's implementation assumes the density is given by 
$$
u_t + \frac{1}{2}(u^2)_x = 0.
$$
In order to transform our state equations into this form, we take the transformation described in the Method of Characteristics section, setting 
\begin{align*}
    r_1 &= V_1\left(1 - \frac{2\rho_1}{P_1}\right) \\
    r_2 &= V_2\left(1 - \frac{2\rho_2}{P_2}\right) 
\end{align*}
so that
\begin{align*}
    \frac{d}{dt}r_1 + r_1\frac{d}{dx}r_1 + \frac{1}{P_1}(V_1 - r_1)\frac{d}{dt}P_1 &= 0
    \\
    \frac{d}{dt}r_2 + r_2\frac{d}{dx}r_2 &= 0.
\end{align*}
In order to use the package as implemented, we needed to make the additional assumption that $\frac{d}{dt}P_1 = 0$ almost everywhere. This assumption is not terribly unrealistic, however, since lanes are typically either closed or open, so lane closure is not only piecewise constant, but discrete-valued. For the results from this package, we assume that there are five lanes on the main road, each capable of holding up to 10 cars / mile and as a result $P_1(t)\in \{0, 10, 20, 30, 40, 50\}$, with $P_1(t)$ constant except on a set of measure zero. We also allow for $P_1 = 0$ by setting the incoming traffic to 0 and keeping the most recent non-zero value of $P_1$ in the denominator. This corresponds to construction workers closing the entrance but allowing cars already on the road to leave like normal.

In [55]:
from matplotlib import pyplot as plt
from clawpack.visclaw import ianimate
from clawpack import pyclaw
from clawpack import riemann
import numpy as np
from IPython.display import clear_output as clear

# This function transforms r1 and r2 back into rho1 and rho2
def r_to_rho(r, v, p):
    return p/2 * (1 - r / v)

# Our cost functional J in terms of r1, r2 and p1. We will use this to evaluate how good a proposed solution is
def cost(x_step, tf, r1, r2, p1, p2=75, v1=50, v2=50/3, c=100):
    return (r_to_rho(r1, v1, p1) + r_to_rho(r2, v2, p2)).sum()*x_step + c*tf

controller = pyclaw.Controller()  # Controller objects do most of the heavy lifting, but need a lot of initialization
controller.tfinal = 0.03
controller.num_output_times = 100

riemann_solver = riemann.burgers_1D
controller.solver = pyclaw.ClawSolver1D(riemann_solver)
controller.solver.all_bcs = pyclaw.BC.extrap  # This allows for non-zero velocities through the boundary
controller.solver.dt = 0.001
controller.solver.max_steps = 1_000_000_000

domain = pyclaw.Domain((0, ), (1, ), (300, ))  # set up the x domain
controller.solution = pyclaw.Solution(controller.solver.num_eqn, domain)

# Set initial data
q = controller.solution.q
xx = domain.grid.p_centers
q[0] = 50 * (1 - 2*np.exp(-(xx[0] - 0.5)**2)/100) # gaussian initial condition

controller.keep_copy = True      # Keep solution data in memory for plotting
controller.output_format = None  # Don't write solution data
controller.solver.dt_initial = 1e99
status = controller.run()


clear()

In [56]:
# Reshape each frame to be in terms of rho rather than r
for i in range(len(controller.frames)):
    r = controller.frames[i]
    r.q = r_to_rho(r.q, 50, 50)

In [57]:
ianimate.ianimate(controller)

2024-04-18 15:26:39,883 INFO CLAW: Animation.save using <class 'matplotlib.animation.HTMLWriter'>


# Interpretation

# Conclusion